In [10]:
# Cleanup any previous Spark versions
!rm -rf spark-*

# Install Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and extract Spark
spark_version = 'spark-3.5.1'
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!ls -l /content/spark-3.5.1-bin-hadoop3

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.5.1-bin-hadoop3"
os.environ["PYTHONPATH"] = f"/content/spark-3.5.1-bin-hadoop3/python:/content/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip"
os.environ["PATH"] += f":{os.environ['SPARK_HOME']}/bin:{os.environ['SPARK_HOME']}/sbin"

# Verify the environment setup
print("JAVA_HOME:", os.environ["JAVA_HOME"])
print("SPARK_HOME:", os.environ["SPARK_HOME"])
print("PYTHONPATH:", os.environ["PYTHONPATH"])
print("PATH:", os.environ["PATH"])

# Install and initialize findspark
!pip install -q findspark
import findspark
findspark.init()

# Create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Home_Sales").getOrCreate()

# Check Spark version to verify installation
print("Spark Version:", spark.version)


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
total 156
drwxr-xr-x 2 1000 1000  4096 Feb 15 11:36 bin
drwxr-xr-x 2 1000 1000  4096 Feb 15 11:36 conf
drwxr-xr-x 6 1000 1000  4096 Feb 15 11:36 data
drwxr-xr-x 4 1000 1000  4096 Feb 15 11:36 examples
drwxr-xr-x 2 1000 1000 20480 Feb 1

In [11]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
#spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [14]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [15]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4_bedroom = spark.sql("""
    SELECT
        year(date) AS year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        year(date)
    ORDER BY
        year
""")
avg_price_4_bedroom.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3_bed_3_bath = spark.sql("""
    SELECT
        date_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        date_built
    ORDER BY
        date_built
""")
avg_price_3_bed_3_bath.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [18]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_specific_homes = spark.sql("""
    SELECT
        date_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY
        date_built
    ORDER BY
        date_built
""")
avg_price_specific_homes.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [19]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_per_view = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        avg_price DESC
""")
avg_price_per_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.006117343902588 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_price_per_view_cached = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        avg_price DESC
""")
avg_price_per_view_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 2.0402724742889404 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("/content/home_sales_partitioned")

In [24]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("/content/home_sales_partitioned")

In [25]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [26]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
avg_price_per_view_parquet = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        parquet_home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        avg_price DESC
""")
avg_price_per_view_parquet.show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.1149895191192627 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False